In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd

In [2]:
# Generating dataset with missing values
df = pd.DataFrame({
    "cgpa": np.random.randn(10),
    "iq": np.random.randn(10) * 10,
    "gender": np.array([np.random.choice([1, 2, 3]) for i in range(10)]),
    "city": np.array([np.random.choice([1, 2, 3]) for i in range(10)])
})

df.loc[5, "cgpa"] = None
df.loc[5, "gender"] = None
df

,cgpa,iq,gender,city
0,-1.631786,8.727138,3.0,2
1,0.844845,-21.058380,2.0,2
2,-0.977704,11.029045,1.0,3
3,-2.148713,-0.958553,1.0,3
4,1.375159,0.611437,3.0,2
5,NaN,-10.229186,NaN,3
6,0.783612,6.303247,3.0,2
7,-0.347020,5.238897,2.0,3
8,-1.066622,-5.853879,1.0,1
9,1.187130,8.233057,1.0,3


In [3]:
# Training Model
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(df.drop(columns="city"), df["city"])

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


*The decision tree is perfectly trained even after dataset having a missing values*


---

### High-Level Analogy: The Substitute Teacher

Imagine a school rule: "If the main teacher is absent, the substitute teacher with the most similar teaching style takes over."

*   **Main Teacher** = The primary, best split (e.g., `Age ≤ 30`).
*   **Missing Data** = The main teacher is absent for a student.
*   **Surrogate Splits** = The list of substitute teachers, ranked by how closely they mimic the main teacher's style.
*   **Process**: For a student with missing `Age`, we go down the list of surrogates (e.g., `Income > $50k`, then `Years_of_Experience > 5`) until we find a feature that *is* present. We then use that split rule.

CART automates this process of finding the best "substitute splits."

---

### The Detailed Process with Math

We'll use a toy dataset. The target is to predict if someone will `Buy_a_Computer`.

| ID | Age       | Income   | Student? | Credit_Rating | Buys_Computer |
|----|-----------|----------|----------|---------------|---------------|
| 1  | ≤30       | High     | No       | Fair          | No            |
| 2  | ≤30       | High     | No       | Excellent     | No            |
| 3  | 31...40   | High     | No       | Fair          | Yes           |
| 4  | >40       | Medium   | No       | Fair          | Yes           |
| 5  | >40       | Low      | Yes      | Fair          | Yes           |
| 6  | >40       | Low      | Yes      | Excellent     | No            |
| 7  | 31...40   | Low      | Yes      | Excellent     | Yes           |
| 8  | ≤30       | Medium   | No       | Fair          | No            |
| 9  | ≤30       | Low      | Yes      | Fair          | Yes           |
| 10 | >40       | Medium   | Yes      | Fair          | Yes           |
| 11 | ≤30       | Medium   | Yes      | Excellent     | Yes           |
| 12 | 31...40   | Medium   | No       | Excellent     | Yes           |
| 13 | 31...40   | High     | Yes      | Fair          | Yes           |
| 14 | >40       | Medium   | No       | Excellent     | No            |
| 15 | **?**     | Medium   | Yes      | Fair          | **?**         |

Our instance (ID 15) has a missing value for `Age`. How does CART decide where to send it?

#### Step 1: Find the Primary Split (Ignoring Missingness)

First, the algorithm finds the best split at the node using all complete data. Suppose it determines that the best split is:
**Primary Split: `Age?`**
*   If `Age ≤ 30` or `Age is 31...40`, send LEFT.
*   If `Age > 40`, send RIGHT.

This split is chosen because it maximizes the reduction in Gini Impurity (or minimizes variance for regression). Let's say the impurity reduction for this split is `ΔI_primary`.

#### Step 2: Find and Rank Surrogate Splits

This is the core of the process. For every other feature (`Income`, `Student?`, `Credit_Rating`), CART asks: "How well does a split on this feature mimic the primary split on `Age`?"

It measures this using a metric called **Similarity or Concordance**.

**The Math: Measuring Similarity of Splits**

The similarity between the primary split `s_p` and a candidate surrogate split `s_s` is measured by the number of instances where both splits send the data point to the same branch.

We can formalize this using a **contingency table**:

| | Surrogate Split sends LEFT | Surrogate Split sends RIGHT | Total |
| :--- | :--- | :--- | :--- |
| **Primary Split sends LEFT** | A | B | P_L |
| **Primary Split sends RIGHT** | C | D | P_R |
| **Total** | S_L | S_R | N |

*   `A + D` are the instances where the splits **agree**.
*   `B + C` are the instances where they **disagree**.

The similarity (or predictive agreement) is defined as:
$$\text{Similarity}(s_p, s_s) = \frac{A + D}{N}$$
This is simply the **proportion of cases where the surrogate split makes the same decision as the primary split**.

However, a more robust measure used in practice is to weight this similarity by the **impurity reduction of the surrogate split itself**. A surrogate that is both similar *and* a powerful splitter in its own right is preferred.

**CART evaluates all possible splits on all other features and chooses the one with the highest similarity score.** This is the **first surrogate split**.

The process continues: "What is the next best surrogate, given the first one?" These surrogates are found to be orthogonal (non-redundant) to the primary and previous surrogates. They are ranked by their similarity score.

**Result:**
1.  **1st Surrogate:** `Student?` (Similarity = 0.85)
    *   If `Yes`, send LEFT (like young/middle age).
    *   If `No`, send RIGHT (like senior age).
2.  **2nd Surrogate:** `Income` (Similarity = 0.70)
    *   If `Low` or `Medium`, send LEFT.
    *   If `High`, send RIGHT.
3.  **3rd Surrogate:** `Credit_Rating` (Similarity = 0.55)
    *   If `Fair`, send LEFT.
    *   If `Excellent`, send RIGHT.

#### Step 3: Handling the Missing Data Point (Instance ID 15)

Now, for our new instance where `Age` is missing:
1.  **Check Primary Split:** `Age` is missing → Cannot use it.
2.  **Check 1st Surrogate:** `Student?` = `Yes` → Decision: Send LEFT.
3.  Since we found a valid surrogate, we do not need to check the others.

The instance is sent down the LEFT branch based on the surrogate feature `Student?`. The process continues at the next node with its own set of primary and surrogate splits.

#### Step 4: What if ALL Surrogates are Missing?

It is possible, though rare, that an instance is missing the primary feature *and* all surrogate features. In this case, CART uses a **fallback mechanism**:

The instance is sent to **both child branches**. As it moves down the tree, its prediction is given a **weight** based on the proportion of training data that went to each branch at the split. Finally, at the leaf, the weighted predictions from all paths are combined.

This is effectively a sophisticated form of probabilistic imputation using the tree's own structure.

---

### Why is this "Perfect" Handling?

This method is considered superior to simple imputation (mean/median/mode) for several reasons:

1.  **Uses Relationships:** It doesn't just fill in a value; it leverages the correlation between features. The surrogates are chosen precisely because they are proxies for the primary split.
2.  **No Information Loss:** Unlike dropping rows with missing values, it uses all available information from the instance.
3.  **No Introduction of Bias:** Simple imputation can distort the distribution of a feature. This method avoids that.
4.  **Embedded in the Model:** The surrogates are part of the trained model itself. There's no separate imputation step that could cause data leakage.

### Summary in a Nutshell

| Step | Action | Key Metric |
| :--- | :--- | :--- |
| **1** | Find the best primary split on feature `X`. | Max Impurity Reduction |
| **2** | For all other features, find the split that best replicates the primary split. Rank them. | `Similarity = (A + D) / N` |
| **3** | For a new instance missing `X`, use the highest-ranked surrogate feature that is not missing. | N/A |
| **4** | If all surrogates are missing, send the instance down all paths with weights. | Proportion of training data |

This elegant approach allows CART to handle real-world, messy data with minimal preprocessing, making it a robust and powerful algorithm.